In [1]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
import json
sys.path.append('/home/kvulic/Vulic/cmos_toolbox_w_spike_sorter/')
#from src.utils.logger_functions import console
from src.cmos_plotter.Latency_calculator import *
from src.utils.metadata_functions import load_metadata_as_dataframe
import logging
logging.getLogger('matplotlib.font_manager').setLevel(logging.ERROR)

In [2]:
MAIN_PATH = f'/itet-stor/kvulic/neuronies/single_neurons/1_Subprojects/Small_Network_Plasticity/3_Processed_Data/BCM_Test/'

In [3]:
metadata = load_metadata_as_dataframe(file_path = os.path.join(MAIN_PATH, ''))

2025-03-24 21:37:33,948 Metadata successfully loaded. 
 First 5 rows: 
                                             Filename  DIV  Chip_ID  Network_ID
0  ID1821_3_DIV45_DATE20240405_1455_spontaneous_b...   45     1821           3
1  ID1821_2_DIV45_DATE20240405_1054_spontaneous_b...   45     1821           2
2  ID1821_1_DIV45_DATE20240405_0849_spontaneous_a...   45     1821           1
3  ID1821_3_DIV45_DATE20240405_1508_spontaneous_a...   45     1821           3
4  ID1821_2_DIV45_DATE20240405_1206_spontaneous_a...   45     1821           2


In [4]:
PROCESSED_DATA_PATH = os.path.join(MAIN_PATH, f'Sorters_old_version/')
#filenames = [f for f in os.listdir(PROCESSED_DATA_PATH) if f.endswith('.pkl')]
#a = 0
for filename in metadata['Filename']:
    chip_id = int(filename.split('_')[0].replace('ID', ''))
    div = int(filename.split('_')[2].replace('DIV', ''))
    #try:
    if filename == 'ID1821_2_DIV45_DATE20240405_1221_spontaneous_before_BCMFreq_Freq5.raw.h5':
        data = np.load(os.path.join(PROCESSED_DATA_PATH, f'{filename[:-3]}_processed.pkl'), allow_pickle=True)
        spikes_extremum = pd.DataFrame(data['SPIKEMAT_EXTREMUM'])
        spikes = np.array([[int(row[0]), float(row[1]), float(row[2])] for row in data['SPIKEMAT']])
        PAIRINGS_PATH = os.path.join(MAIN_PATH, f'biTE_new/')
        OUTPUT_PATH = os.path.join(MAIN_PATH,f'Latency_plots/Latency_plots_TE_new')
        if not os.path.exists(OUTPUT_PATH):
            os.makedirs(OUTPUT_PATH)

        SORTER_PATH = PROCESSED_DATA_PATH
        FULL_SORTER_PATH = os.path.join(SORTER_PATH,'Sorter_'+filename)


        area = filename.split('_')[1]
        # Open the file in binary mode
        sys.modules['numpy.rec'] = np.rec
        #with open(os.path.join(PAIRINGS_PATH, f'{filename[:-3]}_processed_info_metrics.pkl'), 'rb') as f:
        #    data_te = np.array(pickle.load(f))
        data_te = np.load(os.path.join(PAIRINGS_PATH, f'{filename[:-3]}_processed_info_metrics.pkl'), allow_pickle=True)
        exp_duration = data_te['EXPERIMENT_DURATION']
        te_unit_pairs = pd.DataFrame(data_te['mTE'])
        pairings = te_unit_pairs

        with open(os.path.join(FULL_SORTER_PATH, 'wf_folder_curated/sparsity.json'), 'r') as file:
            sorting_info = json.load(file)

        unit_ids = sorting_info['unit_ids']

        spikes, electrodes_pre_all, electrodes_post_all, pre_extremum_all, post_extremum_all, unit_pre_all, unit_post_all, lag_all = get_electrode_unit_info_te(data, pairings, area, unit_ids)
        run_for_all_files_latency_calculation(PAIRINGS_PATH, filename,data_te,exp_duration,unit_ids,spikes,spikes_extremum, pre_extremum_all, post_extremum_all, unit_pre_all, unit_post_all, lag_all)
        #run_for_all_files_latency_plot(OUTPUT_PATH, filename, data_te, exp_duration, unit_ids, spikes, spikes_extremum, pre_extremum_all, post_extremum_all, unit_pre_all, unit_post_all, lag_all)
    #except Exception as e:
    #    print(e)
    #    pass

this happens
Saved extended processed info metrics for ID1821_2_DIV45_DATE20240405_1221_spontaneous_before_BCMFreq_Freq5.raw.h5


In [5]:
filename = 'ID1821_2_DIV45_DATE20240405_1221_spontaneous_before_BCMFreq_Freq5.raw.h5' 
PAIRINGS_PATH = os.path.join(MAIN_PATH, f'biTE_new/')

data_te = np.load(os.path.join(PAIRINGS_PATH, f'{filename[:-3]}_processed_info_metrics.pkl'), allow_pickle=True)
a = data_te['validated_results']

In [8]:
if 'validated_results' in data_te.keys():
    print('yes')

yes


In [6]:
if 'validation_results' in data_te:
    print('yes')


In [9]:
a

[{'source_electrode': 7424,
  'target_electrode': 7424,
  'source_unit_id': 9,
  'target_unit_id': 19,
  'lag': 1.0,
  'latency_extremum': array([(   0,    830.45, 0.  , 'input'),
         (   0,    831.95, 1.5 , 'output'),
         (   1,    878.  , 0.  , 'input'), ...,
         (1069, 197846.35, 0.  , 'input'),
         (1069, 197847.9 , 1.55, 'output'),
         (1070, 199135.  , 0.  , 'input')],
        dtype=[('input spike', '<i4'), ('spike time', '<f8'), ('latency', '<f4'), ('category', '<U6')]),
  'validation': 'good',
  'mTE': 0.013047112,
  'local_mTE': array([[0.0030791],
         [0.0030791],
         [0.0030791],
         ...,
         [0.0030791],
         [0.0030791],
         [0.0030791]])},
 {'source_electrode': 16913,
  'target_electrode': 7424,
  'source_unit_id': 17,
  'target_unit_id': 19,
  'lag': 6.0,
  'latency_extremum': array([(  0, 1.1395000e+02, 0.  , 'input'),
         (  1, 1.5560500e+03, 0.  , 'input'),
         (  2, 1.5842500e+03, 0.  , 'input'),
       

In [7]:
np.where(data_te['LOCAL_mTE'] == data_te['UNIT_IDS'].index(3))

/tmp/ipykernel_1643/4009781825.py:1: DeprecationWarning: Calling nonzero on 0d arrays is deprecated, as it behaves surprisingly. Use `atleast_1d(cond).nonzero()` if the old behavior was intended. If the context of this warning is of the form `arr[nonzero(cond)]`, just use `arr[cond]`.
  np.where(data_te['LOCAL_mTE'] == data_te['UNIT_IDS'].index(3))


(array([], dtype=int64),)

In [8]:
mask = (data_te['LOCAL_mTE']['target'] == data_te['UNIT_IDS'].index(3)) & \
       (data_te['mTE']['source'].astype(int) == data_te['UNIT_IDS'].index(22))

KeyError: 'target'

In [ ]:
data_te['UNIT_IDS'].index(9), data_te['UNIT_IDS'].index(19)

(9, 19)

In [ ]:
data_te['LOCAL_mTE']
array_result = data_te['LOCAL_mTE'][data_te['UNIT_IDS'].index(19)][data_te['UNIT_IDS'].index(9)]
array_result

array([[0.0030791],
       [0.0030791],
       [0.0030791],
       ...,
       [0.0030791],
       [0.0030791],
       [0.0030791]])

In [ ]:
mask = (data_te['mTE']['target'].astype(int) == data_te['UNIT_IDS'].index(3)) & \
       (data_te['mTE']['source'].astype(int) == data_te['UNIT_IDS'].index(22))

In [ ]:
data_te['mTE']['te'][mask]

array([], dtype=float32)

In [ ]:
data_te = np.load(os.path.join(PAIRINGS_PATH, f'ID1821_1_DIV45_DATE20240405_0933_spontaneous_before_BCMFreq_Freq5.raw_processed_info_metrics.pkl'), allow_pickle=True)

In [ ]:
pd.DataFrame(data_te['mTE'])

,target,source,lag,te,te_pval,omnibus_te,omnibus_pval,te_source,target_entropy,norm_te,norm_omnibus_te
0,3.0,20.0,1.0,0.000521,0.005,0.000657,0.005,0.000521,0.032630,0.015958,0.020135
1,3.0,9.0,8.0,0.000081,0.005,0.000657,0.005,0.000081,0.032630,0.002472,0.020135
2,3.0,7.0,3.0,0.000066,0.005,0.000657,0.005,0.000066,0.032630,0.002036,0.020135
3,5.0,11.0,3.0,0.015494,0.005,0.015494,0.005,0.015494,0.042442,0.365067,0.365067
4,6.0,19.0,7.0,0.000194,0.005,0.000194,0.005,0.000194,0.051119,0.003798,0.003798
...,...,...,...,...,...,...,...,...,...,...,...
111,48.0,51.0,3.0,0.000808,0.005,0.001791,0.005,0.000808,0.006080,0.132940,0.294572
112,48.0,30.0,5.0,0.000285,0.005,0.001791,0.005,0.000285,0.006080,0.046945,0.294572
113,48.0,62.0,4.0,0.000040,0.005,0.001791,0.005,0.000040,0.006080,0.006653,0.294572
114,31.0,51.0,2.0,0.000047,0.005,0.000047,0.005,0.000047,0.015778,0.002975,0.002975


In [ ]:
b = pd.DataFrame(data_te['mTE'])
b

,target,source,lag,te,te_pval,omnibus_te,omnibus_pval,te_source,target_entropy,norm_te,norm_omnibus_te
0,3.0,20.0,1.0,0.000521,0.005,0.000657,0.005,0.000521,0.032630,0.015958,0.020135
1,3.0,9.0,8.0,0.000081,0.005,0.000657,0.005,0.000081,0.032630,0.002472,0.020135
2,3.0,7.0,3.0,0.000066,0.005,0.000657,0.005,0.000066,0.032630,0.002036,0.020135
3,5.0,11.0,3.0,0.015494,0.005,0.015494,0.005,0.015494,0.042442,0.365067,0.365067
4,6.0,19.0,7.0,0.000194,0.005,0.000194,0.005,0.000194,0.051119,0.003798,0.003798
...,...,...,...,...,...,...,...,...,...,...,...
111,48.0,51.0,3.0,0.000808,0.005,0.001791,0.005,0.000808,0.006080,0.132940,0.294572
112,48.0,30.0,5.0,0.000285,0.005,0.001791,0.005,0.000285,0.006080,0.046945,0.294572
113,48.0,62.0,4.0,0.000040,0.005,0.001791,0.005,0.000040,0.006080,0.006653,0.294572
114,31.0,51.0,2.0,0.000047,0.005,0.000047,0.005,0.000047,0.015778,0.002975,0.002975


In [ ]:
rel_data = data_te['mTE']['te'][np.where(data_te['mTE']['target'].astype(int) == data_te['UNIT_IDS'].index(3)) and np.where(data_te['mTE']['source'].astype(int) == data_te['UNIT_IDS'].index(22))]
rel_data

array([0.00052072], dtype=float32)

Example STTRP Plotter 

In [ ]:
for idx in range(len(pre_extremum_all)):
        pre_extremum = pre_extremum_all[idx]
        post_extremum = post_extremum_all[idx]
        unit_pre = unit_pre_all[idx]
        unit_post = unit_post_all[idx]
        lag = lag_all[idx]
        unit_to_el = data_te['UNIT_TO_EL']

        latency = plot_latency_and_location_with_extremum_both_plots(OUTPUT_PATH, filename, lag, exp_duration, unit_to_el, pre_extremum, post_extremum, unit_pre, unit_post, unit_ids, spikes, spikes_extremum, extremum_output = post_extremum)
#print(f"Lag {pairings['lag'][idx]}, TE {pairings['te'][idx]}")


ValueError: 7424 is not in list

In [14]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
import json
sys.path.append('/home/kvulic/Vulic/cmos_toolbox_w_spike_sorter/')
#from src.utils.logger_functions import console
from src.cmos_plotter.Plotter_Helper_KV import *
from src.utils.metadata_functions import load_metadata_as_dataframe
import logging
from src.utils.logger_functions import console
import pickle
logging.getLogger('matplotlib.font_manager').setLevel(logging.ERROR)

MAIN_PATH = f'/itet-stor/kvulic/neuronies/single_neurons/1_Subprojects/Small_Network_Plasticity/3_Processed_Data/BCM_Test'
metadata = load_metadata_as_dataframe(file_path = os.path.join(MAIN_PATH, ''))

PAIRINGS_PATH = os.path.join(MAIN_PATH, f'biTE_new/')
OUTPUT_PATH = os.path.join(MAIN_PATH,f'Results')
if not os.path.exists(OUTPUT_PATH):
    os.makedirs(OUTPUT_PATH)

for filename in metadata['Filename']:
    if filename == 'ID1821_2_DIV45_DATE20240405_1221_spontaneous_before_BCMFreq_Freq5.raw.h5':
        console.info(f'Calculating synaptic probability for {filename}...')
        chip_id = int(filename.split('_')[0].replace('ID', ''))
        div = int(filename.split('_')[2].replace('DIV', ''))
        data_te = np.load(os.path.join(PAIRINGS_PATH, f'{filename[:-3]}_processed_info_metrics.pkl'), allow_pickle=True)
        filtered_data = data_te['validated_results']
        filtered_data
        #check if filtered_data has column 'syn probability', if yes, skip
        if 'syn probability' in filtered_data[0].keys():
            print('yes')
        else:
            print('no')

2025-03-24 21:54:57,046 Metadata successfully loaded. 
 First 5 rows: 
                                             Filename  DIV  Chip_ID  Network_ID
0  ID1821_3_DIV45_DATE20240405_1455_spontaneous_b...   45     1821           3
1  ID1821_2_DIV45_DATE20240405_1054_spontaneous_b...   45     1821           2
2  ID1821_1_DIV45_DATE20240405_0849_spontaneous_a...   45     1821           1
3  ID1821_3_DIV45_DATE20240405_1508_spontaneous_a...   45     1821           3
4  ID1821_2_DIV45_DATE20240405_1206_spontaneous_a...   45     1821           2
2025-03-24 21:54:57,047 Calculating synaptic probability for ID1821_2_DIV45_DATE20240405_1221_spontaneous_before_BCMFreq_Freq5.raw.h5...


no
